# Test and compare lineage tracing algorithms on a colony

In [1]:
from bread.data import SegmentationFile, Microscopy, Segmentation
from bread.algo.lineage import LineageGuesserNN, LineageGuesserNearestCell, LineageGuesserBudLum, accuracy
from bread.data import Lineage

In [2]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

## Read data

In [3]:
gt_path = '/Users/farzanehwork/Documents/LPBS/LPBS_track_and_trace_sahand_data/Data_budding_yeast/Recording_external/colony_external_006_lineage_gt.csv'
mask_path = '/Users/farzanehwork/Documents/LPBS/LPBS_track_and_trace_sahand_data/Data_budding_yeast/Recording_external/colony006_segmentation.h5'
fleu_marker_path = '/Users/farzanehwork/Documents/LPBS/LPBS_track_and_trace_sahand_data/Data_budding_yeast/Recording_external/colony006_external_mkate_stack.tif'

# read the ground truth
lin_truth = Lineage.from_csv(gt_path)
# read the segmentation
segmentation = SegmentationFile.from_h5(mask_path).get_segmentation('FOV0')
# read the microscopy
microscopy = Microscopy.from_tiff(fleu_marker_path)['FOV0']

/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/data/_data.py:63: UserWarning: Lineage.confidence initialized with non-float, int64 used.
  warnings.warn(f'Lineage.confidence initialized with non-float, {self.confidence.dtype} used.')


removing small particles with less than 9 pixels that probably have been mis-segmented


## Apply different lineage tracing algorithms

### 1: lineage guessing using neural network (LYN-trace)

In [4]:
# Initialize the LineageGuesserNN
lineage_guesser_nn = LineageGuesserNN(segmentation)

# Guess the lineage using the segmentation data
lineage_guess_lyn_trace = lineage_guesser_nn.guess_lineage()

/Users/farzanehwork/.local/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, preds = torch.max(F.softmax(outputs.data[:,:number_of_candidates]), 1)
/Users/farzanehwork/.local/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, preds = torch.max(F.softmax(outputs.data[:,:number_of_candidates]), 1)
/Users/farzanehwork/.local/lib/python3

In [5]:
# Calculate the accuracy of the guesses
accuracy_score = accuracy(lineage_guess_lyn_trace, lin_truth, strict=False)
print(f'Accuracy of the guesses using LYN-trace: {accuracy_score}')

Accuracy of the guesses using LYN-trace: 0.8537414965986394


### 2: lineage tracing using a budneck markers (LYN-trace-fluo)

In [6]:
# Initialize the Lineage
lineage_guesser_budlum = LineageGuesserBudLum(
    budneck_img=microscopy, segmentation=segmentation)

# Guess the lineage using the segmentation data
lineage_guess_budlum = lineage_guesser_budlum.guess_lineage()

/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:302: NotEnoughFramesWarning: Not enough frames in the movie (bud #238 at frame #113), requested 8, but only 7 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:302: NotEnoughFramesWarning: Not enough frames in the movie (bud #239 at frame #113), requested 8, but only 7 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:302: NotEnoughFramesWarning: Not enough frames in the movie (bud #240 at frame #113), requested 8, but only 7 remaining.
  warnings.warn(NotEnoughFramesWarning(bud_id, time_id, self.num_frames, len(frame_range)))
/Users/farzanehwork/Documents/LPBS/codes/YeaZ-toolbox/src/bread/algo/lineage/_lineage.py:302: 

In [13]:
accuracy_score_budlum = accuracy(lineage_guess_budlum, lin_truth, strict=True)
print(f'Accuracy of the guesses using LYN-trace-fluo: {accuracy_score_budlum}')

Accuracy of the guesses using LYN-trace-fluo: 0.9446153846153846


### 3: lineage tracing by choosing nearest neighbor.

In [8]:
lineage_guesser_nearest_neighbor = LineageGuesserNearestCell(segmentation)

# Guess the lineage using the segmentation data
lineage_guess_nearest_neighbor = lineage_guesser_nearest_neighbor.guess_lineage()

In [9]:
accuracy_score_nearest_neighbor = accuracy(
    lineage_guess_nearest_neighbor, lin_truth, strict=True)
print(
    f'Accuracy of the guesses using nearest neighbor: {accuracy_score_nearest_neighbor}')

Accuracy of the guesses using nearest neighbor: 0.6219512195121951
